<img width="8%" alt="Snowflake.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Snowflake.png" style="border-radius: 15%">

# Snowflake - Ingest json data from local stage
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Snowflake+-+Ingest+json+data+from+local+stage:+Error+short+description">Bug report</a>

**Tags:** #snowflake #data #warehouse #naas_drivers #snippet

**Author:** [Mateusz Polakowski](https://www.linkedin.com/in/polakowski/)

**Last update:** 2023-04-12 (Created: 2022-08-06)

**Description:** This notebook demonstrates how to ingest JSON data from a local stage into Snowflake.

## Input

### Import library

In [1]:
import os
from naas_drivers import snowflake
from snowflake.connector.errors import ProgrammingError

### Setup Snowflake account

If you don't have your SF account, you can easily set up a [30-day trial account with $400 budget here](https://signup.snowflake.com/).

To get your Snowflake account ID essential for connecting, please refer to [Account Identifiers in Snowflake documentation](https://docs.snowflake.com/en/user-guide/admin-account-identifier.html). There are several methods to get your account ID, but the overall rule can be found below:

```<account_identifier>.snowflakecomputing.com```

If you're proceeding with the trial account, it's highly probable that your ID will resemble something like: `xy1234.eu-central-1`.

### Credentials

In [2]:
# Here environment variables are used to pass Snowflake credentials,
# but it's okay to do it in a different manner

sf_username = os.environ["SNOWFLAKE_USER"]
sf_password = os.environ["SNOWFLAKE_PASSWORD"]
sf_account = os.environ["SNOWFLAKE_ACCOUNT"]

## Model

### Connecting to your Snowflake account

In [3]:
snowflake.connect(username=sf_username, password=sf_password, account=sf_account)

### Environment setup

In [4]:
snowflake.database.get_current() is None

In [5]:
snowflake.set_environment(warehouse="COMPUTE_WH", role="ACCOUNTADMIN")

In [6]:
snowflake.database.create("NAAS", or_replace=True, silent=True)
snowflake.database.use("NAAS", silent=True)
snowflake.schema.create("NAAS_SCHEMA", or_replace=True, silent=True)
snowflake.schema.use("NAAS_SCHEMA", silent=True)

In [7]:
snowflake.get_environment()

## Output

### Creating file format

In [8]:
# It's possible to add extra parameters to the function
# They will be passed as additional variables at the end of the query statement
results_ff = snowflake.file_format.create(
    "my_json_format", "JSON", or_replace=True, TRIM_SPACE="TRUE"
)

results_ff

In [9]:
# Let's not forget about `snowflake.execute` functionality
snowflake.execute("SHOW FILE FORMATS;")["results"]

### Creating file format with wrong parameter (not applicable for JSON file format, see [docs](https://docs.snowflake.com/en/sql-reference/sql/create-file-format.html))

In [10]:
try:
    snowflake.file_format.create(
        "my_invalid_son_format", "JSON", or_replace=True, ESCAPE='"\\"'
    )
except ProgrammingError as pe:
    print("Something went wrong!")
    print(pe)

### Dropping file format

In [11]:
# snowflake.file_format.drop('my_json_format', if_exists=True)

### Creating internal stage

In [12]:
snowflake.stage.create(
    stage_name="my_internal_stage", or_replace=True, file_format_name="my_json_format"
)

### Putting data on the stage

In [13]:
snowflake.stage.put(
    filepath="file://~/Downloads/reviews_data.json",
    internal_stage_name="@my_internal_stage",
    silent=True,
)

### Listing files inside stage

In [14]:
result_list_stage = snowflake.stage.list("@my_internal_stage")

print(result_list_stage["results"])
print(result_list_stage["statement"])

### Creating table with query execution

In [15]:
query_create_table = """
    CREATE OR REPLACE TABLE reviews_dev (
      language VARCHAR
      , product_category VARCHAR
      , product_id VARCHAR
      , review_body VARCHAR
      , review_id VARCHAR
      , review_title VARCHAR
      , reviewer_id VARCHAR
      , stars INT
    );
"""

# No worries, Table API will be available soon too!
snowflake.execute(query_create_table)["results"]

### Loading data from internal stage to a table

As we're loading data in JSON format, transformations are required to not put everything into a single VARIANT type column (for more, [see the documentation](https://docs.snowflake.com/en/sql-reference/data-types-semistructured.html)).

In [16]:
transformation_statement = """
    SELECT
        $1:language::varchar AS language
        , $1:product_category::varchar AS product_category
        , $1:product_id::varchar AS product_id
        , $1:review_body::varchar AS review_body
        , $1:review_id::varchar AS review_id
        , $1:review_title::varchar AS review_title
        , $1:reviewer_id::varchar AS reviewer_id
        , $1:stars::int AS stars
    FROM @my_internal_stage
"""

snowflake.copy_into(
    table_name="reviews_dev", source_stage=transformation_statement, silent=True
)

### Data querying straight to Pandas DataFrame

In [17]:
reviews_dev = snowflake.query_pd("SELECT * FROM reviews_dev")
reviews_dev.head()